In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import datetime as dt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import warnings
import os.path
import info
#import lightgbm as lgb
from utils import *

warnings.filterwarnings("ignore")

path_pc = 'C:/Users/admin/Desktop/AI Plan/Finance with AI/Notebooks/'
today = datetime.today()
print("Today's date:", today.strftime("%Y-%m-%d"))

path_img = 'C:/Users/admin/Desktop/AI Plan/Finance with AI/PnL images/'+today.strftime("%Y%m%d")[-6:]+'/'

if not os.path.exists(path_img):
    os.makedirs(path_img)
    print('Created image directory: ')
    print(path_img)

count = 0
day = today
while True:
    path_scoreboard = 'C:/Users/admin/Desktop/AI Plan/Finance with AI/Notebooks/Score Boards individual symbol/'+day.strftime("%Y%m%d")[-6:]+'/'
    if os.path.exists(path_scoreboard+'3988.HK_scoreboard_ensemble.csv') and len(os.listdir(path_scoreboard)) > 0:
        print('\nLatest scoreboard folder: \n', path_scoreboard)
        break
    else:
        day = day - to_days(1)
        count += 1
        if count > 7:
            print("No valid scoreboard path. Please check.")
            break

Today's date: 2020-03-25

Latest scoreboard folder: 
 C:/Users/admin/Desktop/AI Plan/Finance with AI/Notebooks/Score Boards individual symbol/200320/


In [2]:
#path_scoreboard = 'C:/Users/admin/Desktop/AI Plan/Finance with AI/Notebooks/Score Boards individual symbol/200304/'

In [3]:
outcomes_new = load_latest(today, 'outcomes_', path_pc)

Trying  outcomes_2020-03-25.csv
Trying  outcomes_2020-03-24.csv
Loading file:  outcomes_2020-03-24.csv


In [4]:
# outcomes_new = outcomes_new.drop(['log_return_10', 'return_10', 'log_return_5', 'return_5'], axis=1)
outcomes_new = outcomes_new.drop(['log_return_10', 'return_10'#,
                                  #'log_return_5', 'return_5',
                                  #'log_return_4', 'return_4',
                                  #'log_return_3', 'return_3',
                                  #'log_return_2', 'return_2'
                                 ], axis=1)

In [5]:
features_selected = ['(close/ema)-1',
 '(close/sma)-1',
 '1-lower/close',
 #'bbands50_lower',
 'bbands50_lower_slope',
 #'bbands50_upper',
 'bbands50_upper_slope',
 #'close',
 'close_pct_change_5_day',
 'close_scaled50',
 'close_slope',
 #'ema50',
 'ema50_scaled50',
 'ema50_slope',
 #'high',
 'high_scaled50',
 'high_slope',
 'intraday_chg',
 'log volume',
 #'low',
 'low_scaled50',
 'low_slope',
 #'macd',
 #'macd_signal',
 'macd_signal_pct_diff',
 'momentum',
 #'open',
 'open_scaled50',
 'open_slope',
 'rsi14',
 'rsi14_slope',
 #'sma50',
 'sma50_scaled50',
 'sma50_slope',
 'upper/close-1',
 'volatility50',
 'volatility50_ratio',
 #'volume',
 'volume_pct_change_1_day',
 'volume_scaled50',
 'volume_slope',
 'past_return_1',
 'past_return_2',
 'past_return_3',
 'past_return_4',
 'past_return_5',
 'past_return_10',
 'past_log_return_1',
 'past_log_return_2',
 'past_log_return_3',
 'past_log_return_4',
 'past_log_return_5',
 'past_log_return_10'
 ]

In [6]:
last_date = outcomes_new.index.get_level_values('date')[-1].to_pydatetime()
print ("Last date in dataset: ", last_date.strftime("%Y-%m-%d"))

Last date in dataset:  2020-03-24


In [7]:
outcomes_new_dropna = outcomes_new.dropna()

next_day_open = lambda x: x.shift(-1)
outcomes_new_dropna['next_day_open'] = outcomes_new_dropna.groupby(level='symbol').apply(next_day_open)['open']

In [8]:
prob_threshold = 0.7
test_period = 100
test_size = 1
valid_size = 400
training_size = 2000
valid_test_gap = 4
q_upper = 0.9
q_lower = 0.1
# return_col = 'log_return_1'
# return_col_actual = 'return_1'
#return_col = 'log_return_2'
#return_col_actual = 'return_2'
# return_col = 'log_return_3'
# return_col_actual = 'return_3'
return_col = 'log_return_5'
return_col_actual = 'return_5'
# return_col = 'log_return_10'
# return_col_actual = 'return_10'
num_models = 9
random_seeds = [11,22,42,34,45,50,255,135, 229]
max_depth_range = np.arange(2,5,1)
num_leaves_range = np.arange(9,15,1)

In [9]:
from pathlib import Path

#Create new folder for scoreboards
path_scoreboard_new = 'C:/Users/admin/Desktop/AI Plan/Finance with AI/Notebooks/Score Boards individual symbol/'+today.strftime("%Y%m%d")[-6:]+'/'
print("Creating new folder: ", today.strftime("%Y%m%d")[-6:]+'/')
Path(path_scoreboard_new).mkdir(parents=True, exist_ok=True)

Creating new folder:  200325/


In [10]:
master_scoreboard = pd.DataFrame()

import time
start_time = time.time()

for symbol_loop in list(info.multiplier.keys()):
    
    print("-"*80)
    
    print("\nWorking on ", symbol_loop)
    
    outcomes_new_selected_symbol = outcomes_new_dropna.loc[outcomes_new_dropna.index.get_level_values('symbol')==symbol_loop]
    
    score_board_filename = symbol_loop + '_scoreboard_ensemble.csv'
    if os.path.exists(path_scoreboard + score_board_filename):
        score_board_ensemble = pd.read_csv(path_scoreboard + score_board_filename)
    else:
        print("Skipping ", symbol_loop, " as previous data doesn't exist.")
        continue

    #get all trading dates from dataset of this symbol
    data_dates = list(set(outcomes_new_selected_symbol.index.get_level_values(0)))
    data_dates = sorted(data_dates)
    data_converted_dates = []
    for ts in data_dates:
        data_converted_dates.append(ts.to_pydatetime())
    
    #calculate training start dates according to test periods, train, valid and test sizes
    total_num_days = len(data_converted_dates)
    num_periods = int((total_num_days -(valid_size + training_size + valid_test_gap))/test_size)
    start_dates = []
    for i in range(1, num_periods):
        start_dates.append(data_converted_dates[(test_size*-i) - valid_test_gap - valid_size - training_size])
    start_dates = sorted(start_dates)

    #skip symbols that are relatively new and has not enough trading days for training 
    if len(start_dates) < test_period:
        print("Skipped ", symbol_loop, " due to insuffient trading days.")
        continue
    
    #Calculate dates for new test data that are not in scoreboard
    score_board_dates = []
    for date_str in score_board_ensemble['01) start_date']:
        score_board_dates.append(datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S'))
    new_start_dates = list(set(start_dates[-test_period:]) - set(score_board_dates))
    new_start_dates = sorted(new_start_dates)

    if new_start_dates == []:
        print("No new data are found for {}. No backtest is done.".format(symbol_loop))
    else:
        print("Training for ", symbol_loop)

        for start_date in new_start_dates:
            counter = data_converted_dates.index(start_date)
            start_date_valid = data_converted_dates[counter + training_size -1]
            end_date_valid = data_converted_dates[counter + training_size + valid_size -1]
            start_date_test = data_converted_dates[counter + training_size + valid_test_gap + valid_size-1]
            end_date_test = data_converted_dates[counter + training_size + valid_test_gap + valid_size + test_size-1]
            #print("start training date: ", start_date)
            #print("Valid dataset period: ", start_date_valid, " - ", end_date_valid)
            #print("Test datasset period: ", start_date_test, " - ", end_date_test)

            #split into train, valid and test sets
            X_y_train, X_y_valid, X_y_test = train_valid_test_split(outcomes_new_selected_symbol, start_date,
                                                                    start_date_valid, end_date_valid, start_date_test, end_date_test)

            #calculate upper threshold in training set, then create targets for both valid and test
            X_y_train, X_y_valid, X_y_test = add_target_upper(X_y_train, X_y_valid,
                                                              X_y_test, q_upper, 'target', return_col)

            X_y_test['symbol'] = X_y_test.index.get_level_values('symbol')

            X_valid, y_valid = feature_target_split(X_y_valid, features_selected, 'target')
            X_test, y_test = feature_target_split(X_y_test, features_selected, 'target')

            #create 2 extra sets for calculating gain
            X_valid_close = X_y_valid[['close',return_col_actual]]
            X_test_close = X_y_test[['close',return_col_actual, 'symbol','next_day_open']]

            num_shares = []

            for i, row in X_test_close.iterrows():
                num_shares.append(info.board_lots[row[2]] * info.multiplier[row[2]])

            X_test_close['num_shares'] = num_shares

            best_models = []

            #Stage 1 - create multiple lgbm models with various downsampling random seed
            for i in range(num_models):
                #print("Trainint lgb model {}".format(str(i)))

                #downsample the training set's negative data points
                X_y_train_resampled = downsample(X_y_train, 'target', test_ratio=0.11, random_seed=random_seeds[i])

                #split into features and target sets
                X_train, y_train = feature_target_split(X_y_train_resampled, features_selected, 'target')
                (best_model, best_pres_model, max_total_gain, optimal_depth, optimal_num_leaves, max_precision,
                 optimal_precision_depth, optimal_precision_num_leaves,
                 max_precision_total_gain) = lgb_train(X_train, y_train, X_valid, y_valid, X_valid_close, max_depth_range, num_leaves_range, return_col_actual,
                                                 min_data = 11, metric = 'auc', prob_threshold = prob_threshold)   
                #best_pres_models.append(best_pres_model)
                best_models.append(best_model)

            #Stage 2 - Generate predictions for valid set from multi-lgb models
            valid_df, cols = multi_lgb_predict(best_models, X_valid, y_valid)

            #Use predictions to train logistic regression
            clf = LogisticRegression(class_weight = "balanced", solver = 'liblinear').fit(valid_df[cols], valid_df['target'])

            #generate out-of-sample predictions
            test_df, cols = multi_lgb_predict(best_models, X_test, y_test)
            class_pred = pd.DataFrame(data = clf.predict(test_df[cols]), columns = ['pred'])
            y_class_pred = class_pred.set_index(X_test.index)

            #calculate test gain using ensemble model
            actual_gain, total_amount_spent = ensemble_actual_gain(y_class_pred, X_test_close, return_col_actual, sign=1)
            score_board_ensemble = score_board_ensemble.append({'01) start_date': start_date,
                                                                '02) start_date_valid':start_date_valid,
                                                                '03) start_date_test': start_date_test,
                                                                '04) actual_gain': actual_gain,
                                                                '05) total_amount_spent': total_amount_spent}, ignore_index=True)

        score_filename = symbol_loop + '_scoreboard_ensemble.csv'
        print("Saving score board for ", symbol_loop)
        save_csv(score_board_ensemble, path_scoreboard_new, score_filename)

        gain_loss_ratio = gain_vs_loss(score_board_ensemble['04) actual_gain'])

        master_scoreboard = master_scoreboard.append({'symbol': symbol_loop,
                                                      'Gain Loss Raio': gain_loss_ratio,
                                                      'Test Total Gain': score_board_ensemble['04) actual_gain'].sum(),
                                                      'Profit Percentage': score_board_ensemble['04) actual_gain'].sum()/(score_board_ensemble['05) total_amount_spent'].mean()*5),
                                                      'Total amount spent': score_board_ensemble['05) total_amount_spent'].mean()}, ignore_index=True)

        print("Saving plots for ", symbol_loop)
        plot_filename_total = symbol_loop + '_ensemble_total.png'
        plot_filename_pres = symbol_loop + '_ensemble_pres.png'

        fig_total = score_board_ensemble['04) actual_gain'].cumsum().plot().get_figure()
        fig_total.savefig(path_img+plot_filename_total)

        #fig_pres.clf()
        fig_total.clf()

print("--- %s seconds ---" % (time.time() - start_time))
print(str(dt.timedelta(seconds=(time.time() - start_time))))

--------------------------------------------------------------------------------

Working on  0001.HK
Training for  0001.HK
Saving score board for  0001.HK
Saving plots for  0001.HK
--------------------------------------------------------------------------------

Working on  0002.HK
Training for  0002.HK
Saving score board for  0002.HK
Saving plots for  0002.HK
--------------------------------------------------------------------------------

Working on  0003.HK
Training for  0003.HK
Saving score board for  0003.HK
Saving plots for  0003.HK
--------------------------------------------------------------------------------

Working on  0005.HK
Training for  0005.HK
Saving score board for  0005.HK
Saving plots for  0005.HK
--------------------------------------------------------------------------------

Working on  0006.HK
Training for  0006.HK
Saving score board for  0006.HK
Saving plots for  0006.HK
--------------------------------------------------------------------------------

Working 

Saving score board for  2388.HK
Saving plots for  2388.HK
--------------------------------------------------------------------------------

Working on  2628.HK
Training for  2628.HK
Saving score board for  2628.HK
Saving plots for  2628.HK
--------------------------------------------------------------------------------

Working on  3328.HK
Training for  3328.HK
Saving score board for  3328.HK
Saving plots for  3328.HK
--------------------------------------------------------------------------------

Working on  3988.HK
Training for  3988.HK
Saving score board for  3988.HK
Saving plots for  3988.HK
--- 3718.387861251831 seconds ---
1:01:58.387861


In [11]:
save_csv(master_scoreboard, path_pc, 'master_scoreboard_ensemble_'+last_date.strftime("%Y-%m-%d")+'.csv')

In [12]:
master_scoreboard = pd.read_csv(path_pc+'master_scoreboard_ensemble_'+last_date.strftime("%Y-%m-%d")+'.csv')
master_scoreboard

,Unnamed: 0,Gain Loss Raio,Profit Percentage,Test Total Gain,Total amount spent,symbol
0,0,0.304632,-0.600852,-31900.0,10618.248175,0001.HK
1,1,1.304000,0.052505,3800.0,14474.817518,0002.HK
2,2,1.098039,0.018871,1200.0,12718.248175,0003.HK
3,3,0.768765,-0.117228,-3820.0,6517.226277,0005.HK
4,4,0.473611,-0.213763,-9475.0,8864.963504,0006.HK
5,5,0.242221,-0.521745,-31660.0,12136.204380,0011.HK
6,6,0.750439,-0.127764,-7100.0,11114.233577,0012.HK
7,7,0.613158,-0.202199,-22050.0,21810.218978,0016.HK
8,8,0.880744,-0.056747,-3270.0,11524.817518,0017.HK
9,9,0.343644,-0.518592,-38725.0,14934.671533,0019.HK


In [13]:
master_scoreboard.sort_values(by=['Test Total Gain'], ascending=False)[0:5]

,Unnamed: 0,Gain Loss Raio,Profit Percentage,Test Total Gain,Total amount spent,symbol
31,31,1.579301,0.289510,17240.0,11909.798658,1093.HK
26,26,2.050847,0.270512,6200.0,4583.892617,0939.HK
21,21,1.888179,0.260229,5560.0,4273.154362,0700.HK
20,20,1.157738,0.085563,5300.0,12388.590604,0688.HK
29,29,1.206660,0.097103,5275.0,10864.765101,1044.HK


In [14]:
master_scoreboard.sort_values(by=['Profit Percentage'], ascending=False)[0:5]

,Unnamed: 0,Gain Loss Raio,Profit Percentage,Test Total Gain,Total amount spent,symbol
31,31,1.579301,0.289510,17240.0,11909.798658,1093.HK
26,26,2.050847,0.270512,6200.0,4583.892617,0939.HK
21,21,1.888179,0.260229,5560.0,4273.154362,0700.HK
29,29,1.206660,0.097103,5275.0,10864.765101,1044.HK
20,20,1.157738,0.085563,5300.0,12388.590604,0688.HK


In [15]:
master_scoreboard.sort_values(by=['Gain Loss Raio'], ascending=False)[0:5]

,Unnamed: 0,Gain Loss Raio,Profit Percentage,Test Total Gain,Total amount spent,symbol
26,26,2.050847,0.270512,6200.0,4583.892617,0939.HK
21,21,1.888179,0.260229,5560.0,4273.154362,0700.HK
31,31,1.579301,0.289510,17240.0,11909.798658,1093.HK
1,1,1.304000,0.052505,3800.0,14474.817518,0002.HK
29,29,1.206660,0.097103,5275.0,10864.765101,1044.HK


In [16]:
master_scoreboard.loc[master_scoreboard['Gain Loss Raio'] == -float("inf")]

,Unnamed: 0,Gain Loss Raio,Profit Percentage,Test Total Gain,Total amount spent,symbol
